In [1]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt


from sklearn.ensemble import IsolationForest
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans


from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.holtwinters import ExponentialSmoothing
from statsmodels.tsa.arima.model import ARIMA

import pandas as pd
import numpy as np
from statsmodels.tsa.holtwinters import ExponentialSmoothing
import matplotlib.pyplot as plt


file_path = '/Users/yunustopcu/Documents/GitHub/predictive-solar-supply-chain/power_usage_2016_to_2020.csv'
data = pd.read_csv(file_path)



In [2]:
from datetime import datetime

# Define a function to check if the date is in the expected format
def check_date_format(date_string):
    try:
        # If the date can be parsed in the expected format, it's consistent
        datetime.strptime(date_string, '%Y-%m-%d %H:%M:%S')
        return True
    except ValueError:
        # If parsing fails, the format is inconsistent
        return False

# Apply the check to the 'StartDate' column
date_format_consistency = data['StartDate'].apply(check_date_format)

# Check if there are any False values in the results, indicating inconsistencies
inconsistencies = not date_format_consistency.all()

# Count the number of inconsistent dates, if any
inconsistent_dates_count = len(data[~date_format_consistency])

inconsistencies, inconsistent_dates_count


(False, 0)

In [3]:
# To find the first and last date in the dataset, we'll convert the 'StartDate' column to datetime format and then find the minimum and maximum dates.

# Convert 'StartDate' to datetime format
#data['StartDate'] = pd.to_datetime(data['StartDate'])

n = data.shape[0]
p1 = pd.Series(range(n), pd.period_range('2016-06-01 00:00:00', freq = '1H', periods = n))
data['StartDate'] = p1.to_frame().index


# Find the first (minimum) and last (maximum) date
first_date = data['StartDate'].min()
last_date = data['StartDate'].max()

first_date, last_date

# DEBUGGING LINE, DONT BOTHER
#data.to_csv('data.csv', index=False)

# To trim the dataset to only include data from 2017-01-01 to 2018-01-01, we'll filter the 'StartDate' to be within this range.

start_date = '2017-01-01'
end_date = '2018-01-01'

# Filter the dataset for the given date range
trimmed_data = data[(data['StartDate'] >= start_date) & (data['StartDate'] <= end_date)]

# Check the first few rows to ensure the trimming was successful
trimmed_data.head(), trimmed_data.tail()


# Convert 'StartDate' from PeriodDtype to datetime
trimmed_data['StartDate'] = trimmed_data['StartDate'].dt.to_timestamp()
trimmed_data['StartDate'] = pd.to_datetime(trimmed_data['StartDate'])
trimmed_data.set_index('StartDate', inplace=True)
trimmed_data = trimmed_data.asfreq('H', method='ffill')  # Ensure hourly frequency, filling any missing values


/var/folders/p5/4gqxjwtd7k73wdww84v63q9c0000gn/T/ipykernel_31130/1698577670.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  trimmed_data['StartDate'] = trimmed_data['StartDate'].dt.to_timestamp()
/var/folders/p5/4gqxjwtd7k73wdww84v63q9c0000gn/T/ipykernel_31130/1698577670.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  trimmed_data['StartDate'] = pd.to_datetime(trimmed_data['StartDate'])


In [4]:
trimmed_data.head()


,Value (kWh),day_of_week,notes
StartDate,,,
2017-01-01 00:00:00,0.840,6,weekend
2017-01-01 01:00:00,0.731,6,weekend
2017-01-01 02:00:00,0.254,6,weekend
2017-01-01 03:00:00,0.270,6,weekend
2017-01-01 04:00:00,0.318,6,weekend


In [9]:
power_consumption_data = trimmed_data
power_consumption_data.drop(['day_of_week', 'notes'], axis=1, inplace=True)


In [10]:
power_consumption_data.head(24)

,Value (kWh)
StartDate,
2017-01-01 00:00:00,0.840
2017-01-01 01:00:00,0.731
2017-01-01 02:00:00,0.254
2017-01-01 03:00:00,0.270
2017-01-01 04:00:00,0.318
2017-01-01 05:00:00,0.229
2017-01-01 06:00:00,0.330
2017-01-01 07:00:00,0.240
2017-01-01 08:00:00,0.340


In [33]:
file_path = '/Users/yunustopcu/Documents/GitHub/predictive-solar-supply-chain/GTI_2017.csv'
data = pd.read_csv(file_path)

data.head(24)

data.set_index('date', inplace=True)


In [34]:
gti_data = data

In [35]:
gti_data.head(24)

,GTI
date,
2017-01-01 00:00:00,0.000000
2017-01-01 01:00:00,0.000000
2017-01-01 02:00:00,0.000000
2017-01-01 03:00:00,0.000000
2017-01-01 04:00:00,0.000000
2017-01-01 05:00:00,0.000000
2017-01-01 06:00:00,0.000000
2017-01-01 07:00:00,0.000000
2017-01-01 08:00:00,0.000000


In [36]:
gti_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 8760 entries, 2017-01-01 00:00:00 to 2017-12-31 23:00:00
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   GTI     8760 non-null   float64
dtypes: float64(1)
memory usage: 136.9+ KB


In [37]:
power_consumption_data.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 8761 entries, 2017-01-01 00:00:00 to 2018-01-01 00:00:00
Freq: H
Data columns (total 1 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Value (kWh)  8761 non-null   float64
dtypes: float64(1)
memory usage: 136.9 KB


In [38]:
power_consumption_data.describe()

,Value (kWh)
count,8761.000000
mean,0.768589
std,0.770653
min,0.064000
25%,0.293000
50%,0.457000
75%,0.918000
max,5.607000


In [39]:
gti_data.describe()

,GTI
count,8760.000000
mean,176.989720
std,249.030660
min,0.000000
25%,0.000000
50%,0.000000
75%,322.392442
max,980.798123


In [40]:
gti_data['GTI'] = gti_data['GTI'] * 0.004

In [41]:
gti_data.describe()

,GTI
count,8760.000000
mean,0.707959
std,0.996123
min,0.000000
25%,0.000000
50%,0.000000
75%,1.289570
max,3.923192


In [42]:
hourly_solar_production_data = gti_data

In [43]:

# Assuming hourly_solar_production_data is your DataFrame
hourly_solar_production_data.to_csv('hourly_solar_production_data.csv', index=True)
